# Core

> The Plash CLI tool

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser
from httpx import post as xpost, get as xget
from pathlib import Path
from uuid import uuid4
from time import time, sleep

import io, sys, tarfile

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash.env'

In [ ]:
#| export
def get_global_cfg():
    """Works for all operating systems."""
    try: return parse_env(fn=Path(PLASH_CONFIG_HOME))
    except FileNotFoundError: 
        Path(PLASH_CONFIG_HOME).touch()
    return parse_env(fn=Path(PLASH_CONFIG_HOME))

In [ ]:
#| export
def is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

In [ ]:
#| export
def create_tar_archive(path # Path to directory containing FastHTML app
                      )->io.BytesIO: # Buffer of tar directory
    "Creates a tar archive of a directory, excluding files based on is_included"
    buf = io.BytesIO()
    files = L(Path(path).iterdir()).filter(is_included)

    with tarfile.open(fileobj=buf, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
    buf.seek(0)
    return buf, len(files)

In [ ]:
#| export
def validate_app(path):
    "Validates that the app in the directory `path` is deployable as a FastHTML app"
    print("Analyzing project structure...")

    main_file = Path(path) / "main.py"
    if not main_file.exists():
        print('[red bold]ERROR: Your FastHTML app must have a main.py[/red bold]')
        print(f'Your path is: [bold]{path}[/bold]')
        sys.exit(1)

In [ ]:
#| export
@call_parse
def deploy(
    path:Path=Path('.'), # Path to project
    local:bool=False,  # local dev
    port:int=5002):
    """🚀 Ship your app to production"""
    print('Initializing deployment...')
    validate_app(path)
    tarz, filecount = create_tar_archive(path)

    plash_app = Path(path) / '.plash'
    if not plash_app.exists():
        # Create the .plash file and write the app name
        plash_app.write_text(f'export PLASH_APP_ID=fasthtml-app-{str(uuid4())[:8]}')
    
    aid = parse_env(fn=plash_app)['PLASH_APP_ID']
    cfg = get_global_cfg()
    url = 'https://pla.sh/upload'
    if local: url = f'http://localhost:{port}/upload'
    headers = {'Authorization': f'Bearer {cfg["PLASH_TOKEN"]}'}
    print(f'Uploading {filecount} files...')
    resp = xpost(url, headers=headers, files={'file': tarz}, timeout=300.0,
                data={'aid': aid, 'email': cfg['PLASH_EMAIL']})
    if resp.status_code == 200: 
        print('✅ Upload complete! Your app is currently being built.')
        print(f'It will be live at https://{aid}.pla.sh')
    else:
        print(f'Failure {resp.status_code}')
        print(f'Failure {resp.text}')

In [ ]:
# deploy('test_apps/minimal')

## Oauth login

In [ ]:
#| export
def poll_token(paircode, host, protocol='https', interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    while time()-start < timeout:
        resp = xget(f"{protocol}://{host}/token?paircode={paircode}").raise_for_status()
        if resp.text.strip(): return resp.text
        sleep(interval)
        
@call_parse
def login(
    local:bool=False,  # local dev
    port:int=5002      # port for local development
):
    "Authenticate CLI with server and save token"
    protocol = 'https' if not local else 'http'
    host = 'pla.sh' if not local else f'localhost:{port}'
    paircode = "cli-paircode-" + secrets.token_urlsafe(16)
    url = f'{protocol}://{host}/cli_login?paircode={paircode}'
    
    login_url = xget(url).text
    print(f"Opening browser for authentication:\n{login_url}\n")
    webbrowser.open(login_url)
    
    token = poll_token(paircode, host, protocol=protocol)
    if token:
        token_file = PLASH_CONFIG_HOME.parent / 'plash_auth_token.txt'  # TODO: rename when harmonizing w plash.env
        Path(token_file).write_text(token)
        print(f"Authentication successful! Token saved to {token_file}")
    else: print("Authentication timed out.")

## Export

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()